# Visualizing Field Coverage with a Stacked Graph

The following is an example of how to pull data from our backend to visualize it in this notebook using d3.  This example is not a particularly useful for actually analyzing field coverage, but it does nicely illustrate the flow from our graphql backend to this notebook to a visual using a python library or something like d3js.  Please see the resulting graph as a proof of concept, not a sharable visual.


## Import all Our necessary python dependencies



In [1]:
from gql import gql, Client # generate graphql queries and setup client for talking to graphql endpoints.
from gql.transport.requests import RequestsHTTPTransport # connect our client to a gql endpoint over https
import pandas as pd # for manipulating our data 
import json # parse returned gql into json for our d3 visualization
from IPython.display import display, Javascript, HTML # allow us to use javascript within python 
                                                      # and manipulate the dom of this notebook


## Import Our Data



Luckily, our stable_endpoint_stats gives us exactly that.

In [23]:
stats_query=gql("""
query{
  stable_endpoint_stats {
    test_hits
    conf_hits
    date
  }
}
""")

Our backend is reachable through a single api endpoint, located here:

In [24]:
BACKEND_URL = "https://zz-hasura.sharing.io/v1/graphql"

To be able to post this query to our graphql backend, we need to setup a gql client that fetches from this endpoint.  the python graphql client calls this the `transport`

In [25]:
_transport = RequestsHTTPTransport(
    url=BACKEND_URL,
    use_json=True
)

client = Client(
    transport=_transport,
    fetch_schema_from_transport=True
)

We can now use the client to execute this query.  Printing it out shows that our returned data matches our desired structure.

In [26]:
stable_endpoint_stats = client.execute(stats_query)["stable_endpoint_stats"]
print(json.dumps(stable_endpoint_stats, indent=4, sort_keys=True))

[
    {
        "conf_hits": 118,
        "date": "2019-09-16",
        "test_hits": 171
    },
    {
        "conf_hits": 124,
        "date": "2019-09-30",
        "test_hits": 171
    },
    {
        "conf_hits": 114,
        "date": "2019-10-08",
        "test_hits": 165
    }
]


Now we can visualize it!

## Visualizing the Data

I am using the example provided by Stefan Liipens' excellent blogpost: [Custom D3 Visualization in a Jupyter Notebook](https://www.stefaanlippens.net/jupyter-custom-d3-visualization.html)

First, we tell our notebook to require in d3 (via it's minified file) and our  linechart visualization held in this local notebook folder.

In [27]:
display(Javascript("require.config({paths: {d3: 'https://d3js.org/d3.v3.min'}});"))
display(Javascript(filename="linechart.js"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The barchart visualization is expecting a container and data.  The container is the dom element we'll append our svg to, and then our imported data will generate a bar chart.

element is a global variable indicating the output cell for wherever the code is located, and so by wrapping our javascript in closures, we can keep element local to any particular cell.

So here we are defining a python function, that is just a string interpoloation of a javascript function, that thinks it is latching to an html page, but is actually latching onto a single cell (within an html page).

In [28]:
def draw_chart():
    display(Javascript("""
        (function(element){
            require(['lineChart'], function(lineChart) {
                lineChart(element.get(0), %s);
            });
        })(element);
    """ % json.dumps(stable_endpoint_stats)))

To include the visualization in our notebook, we just call the function.

In [29]:
draw_chart()

<IPython.core.display.Javascript object>